In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [18]:
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(452)
    torch.cuda.manual_seed_all(452)
    
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(132)
np.random.seed(31445)

In [3]:
!nvidia-smi

Sun Nov 27 19:26:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import os

if not os.path.exists("/content/train/"):
  !unzip gdrive/MyDrive/SSNE_proj3/train.zip
  !unzip gdrive/MyDrive/SSNE_proj3/test_all.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: test_all/48404765780136605.JPEG  
  inflating: test_all/5623874037695173.JPEG  
  inflating: test_all/8830923257791682.JPEG  
  inflating: test_all/5999784560329912.JPEG  
  inflating: test_all/641941689724576.JPEG  
  inflating: test_all/9636463870285614.JPEG  
  inflating: test_all/713318944677822.JPEG  
  inflating: test_all/4913615325728824.JPEG  
  inflating: test_all/008735498505776818.JPEG  
  inflating: test_all/3814762033873971.JPEG  
  inflating: test_all/23028839087812902.JPEG  
  inflating: test_all/8989128654123999.JPEG  
  inflating: test_all/6757518822057905.JPEG  
  inflating: test_all/725959409539307.JPEG  
  inflating: test_all/054779581564842195.JPEG  
  inflating: test_all/4833253932036542.JPEG  
  inflating: test_all/8826839449367463.JPEG  
  inflating: test_all/394024301579029.JPEG  
  inflating: test_all/6814583823502609.JPEG  
  inflating: test_all/3444018406073345.JPEG  
  inflating: 

### Base line - 50-kilka%
### Dobre wyniki - 70-kilka%

In [20]:
torch.cuda.set_device(0)
device = torch.device("cuda")

### Wczytywanie danych

In [24]:
import torch.utils.data as data


dataFol = torchvision.datasets.ImageFolder(root="train")
classes = dataFol.classes

train_set_size = int(len(dataFol) * 0.8)
valid_set_size = len(dataFol) - train_set_size

train_set, valid_set = data.random_split(dataFol, [train_set_size, valid_set_size])

valid_set.dataset.transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])

tr = [train_set for i in range(4)]

tr[0].dataset.transform = transforms.Compose([
     transforms.RandomHorizontalFlip(1),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])

tr[1].dataset.transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])

tr[2].dataset.transform = transforms.Compose([
     transforms.RandomRotation(degrees=(-15,15)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])

tr[3].dataset.transform = transforms.Compose([
     transforms.RandomCrop(size=(52,52)),
     transforms.Pad(padding=6, padding_mode="edge"),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])


train_set = torch.utils.data.ConcatDataset([tr[0], tr[1], tr[2], tr[3]])

print(len(train_set) + len(valid_set))

print('='*30)
print('Train data set:', len(train_set))
print('Valid data set:', len(valid_set))

299235
Train data set: 281632
Valid data set: 17603


###Create dataloaders

In [25]:
batch_size=128
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# plt.figure(figsize = (20,10))

# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()
# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))

# print(' '.join('%5s' % train_set.dataset.classes[labels[j]] for j in range(batch_size)))

##Create model class

In [27]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        outChan = [80, 150, 150, 200]
        kerSize = [5, 4, 4, 3]
        pool = [2, 2, 1, 2]
        self.conv1 = nn.Conv2d(3,          outChan[0], kerSize[0], padding=1)
        self.conv_bn1 = nn.BatchNorm2d(outChan[0])
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(outChan[0], outChan[1], kerSize[1], padding=1)
        self.conv_bn2 = nn.BatchNorm2d(outChan[1])
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(outChan[1], outChan[2], kerSize[2], padding=1)
        self.conv_bn3 = nn.BatchNorm2d(outChan[2])

        self.conv4 = nn.Conv2d(outChan[2], outChan[3], kerSize[3], padding=1)
        self.conv_bn4 = nn.BatchNorm2d(outChan[3])
        self.pool4 = nn.MaxPool2d(2)
        s=64
        for i in range(len(outChan)):
          s = (s-kerSize[i]+3)/pool[i]

        self.drop0 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(outChan[-1] *  int(s) ** 2, 5000)
        self.bn1 = nn.BatchNorm1d(5000)
        self.drop1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(5000, 3000)
        self.bn2 = nn.BatchNorm1d(3000)
        self.drop2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(3000, 50)

    def forward(self, x):
        x = self.pool1(F.relu( self.conv_bn1(self.conv1(x) )))
        x = self.pool2(F.relu( self.conv_bn2(self.conv2(x) )))
        x = F.relu( self.conv_bn3(self.conv3(x) ))
        x = self.pool4(F.relu( self.conv_bn4(self.conv4(x) )))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.drop0(x)
        x = self.drop1(F.relu( self.bn1(self.fc1(x)) ))
        x = self.drop2(F.relu( self.bn2(self.fc2(x)) ))
        x = self.fc3(x)
        return x

net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 80, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv_bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(80, 150, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv_bn2): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(150, 150, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv_bn3): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(150, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_bn4): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop0): Dropout(p=0.5, inplace=False)
  (fc

In [ ]:
# checkpoint = torch.load("/content/gdrive/MyDrive/SSNE/Lab7/model_52")
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

In [28]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [29]:
def getAccuarcy(data):
  correct = 0
  total = 0
  net.eval()
  with torch.no_grad():
      for images, labels in data:
          images = images.to(device)
          # calculate outputs by running images through the network 
          outputs = net(images).cpu()
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  return (100 * correct / total)

In [30]:

iters = 15
for epoch in range(iters): 
    torch.save(net.state_dict(), "/content/gdrive/MyDrive/SSNE_proj3/model_67_final")
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('[%d/%d] loss: %.3f' % (epoch+1, iters, running_loss / 2000))
    running_loss = 0.0
    print('Validation acc: %.3f %%' % getAccuarcy(valloader))
print('Finished Training')

[1/15] loss: 2.538
Validation acc: 47.759 %
[2/15] loss: 1.789
Validation acc: 52.042 %
[3/15] loss: 1.445
Validation acc: 58.507 %
[4/15] loss: 1.200
Validation acc: 61.518 %
[5/15] loss: 1.000
Validation acc: 63.307 %
[6/15] loss: 0.851
Validation acc: 64.381 %
[7/15] loss: 0.729
Validation acc: 64.239 %
[8/15] loss: 0.638
Validation acc: 65.523 %
[9/15] loss: 0.570
Validation acc: 65.619 %
[10/15] loss: 0.515
Validation acc: 65.722 %
[11/15] loss: 0.466
Validation acc: 65.716 %
[12/15] loss: 0.434
Validation acc: 66.381 %
[13/15] loss: 0.404
Validation acc: 66.926 %
[14/15] loss: 0.378
Validation acc: 66.273 %
[15/15] loss: 0.359
Validation acc: 66.864 %
Finished Training


In [ ]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaddddddddddddddddddddsssssggggggggggggg ffffffffhhhhhhhhhhhhsssss

In [32]:
print('Validation acc: %.3f %%' % getAccuarcy(valloader))

Validation acc: 66.580 %


In [33]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in valloader:
        images, labels = data    
        images = images.to(device)
        outputs = net(images).cpu()   
        _, predictions = torch.max(outputs, dim=1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
            # print accuracy for each class
for classname, correct_count in sorted(correct_pred.items(), 
                                       key=lambda item: float(item[1]) / total_pred[item[0]]):
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy for class bird  is: 47.5 %
Accuracy for class snake is: 49.9 %
Accuracy for class worm  is: 50.3 %
Accuracy for class carbon is: 51.1 %
Accuracy for class fish  is: 53.6 %
Accuracy for class acoustic is: 54.7 %
Accuracy for class squash is: 55.7 %
Accuracy for class egg   is: 57.2 %
Accuracy for class towel is: 57.3 %
Accuracy for class pot   is: 57.5 %
Accuracy for class crocodilian is: 58.4 %
Accuracy for class corn  is: 58.5 %
Accuracy for class echinoderm is: 59.0 %
Accuracy for class bread is: 59.1 %
Accuracy for class bomb  is: 60.1 %
Accuracy for class icecream is: 60.3 %
Accuracy for class nest  is: 60.3 %
Accuracy for class gauge is: 60.9 %
Accuracy for class crab  is: 60.9 %
Accuracy for class battery is: 61.5 %
Accuracy for class swine is: 61.8 %
Accuracy for class antenna is: 62.3 %
Accuracy for class frog  is: 62.7 %
Accuracy for class motor is: 64.5 %
Accuracy for class cat   is: 65.7 %
Accuracy for class birch is: 66.1 %
Accuracy for class bean  is: 67.9 %
Accur

In [31]:
torch.save(net.state_dict(), "/content/gdrive/MyDrive/SSNE_proj3/model_67_final")

**Predictions**

In [34]:
from PIL import Image
class CustomDataSet(torchvision.datasets.VisionDataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.images = os.listdir(self.main_dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(f"{self.main_dir}/{img_name}").convert("RGB")
        tensor_image = self.transform(img)
        return tensor_image, img_name

In [35]:
test_set = CustomDataSet("test_all", transform=transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ]))

In [36]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [45]:
preds_to_write_csv = []
net.eval()
with torch.no_grad():
    for data in test_loader:
        input, filenames = data
        outputs = net(input.to(device))
        _, predictions = torch.max(outputs, dim=1)
        preds_to_write_csv.append((predictions, filenames))

In [46]:
import csv
with open("/content/gdrive/MyDrive/SSNE_proj3/bielak_nitkiewicz.csv", 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for pred_batch in preds_to_write_csv:
      for pred, filename in zip(pred_batch[0], pred_batch[1]):
          csv_writer.writerow([filename, int(pred)])